<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
FILEPATH = ('./../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [2]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [3]:
df = pd.read_csv(FILEPATH)
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [5]:
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg")

tokenizer = Tokenizer(nlp.vocab)

In [11]:
STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [12]:
df['tokens'] = df['reviews.text'].apply(tokenize)

In [13]:
df[['reviews.text', 'tokens']].head()

,reviews.text,tokens
0,I order 3 of them and one of the item is bad q...,"[order, item, bad, quality, missing, backup, s..."
1,Bulk is always the less expensive way to go fo...,"[bulk, expensive, way, products, like]"
2,Well they are not Duracell but for the price i...,"[duracell, price, happy]"
3,Seem to work as well as name brand batteries a...,"[work, brand, batteries, better, price]"
4,These batteries are very long lasting the pric...,"[batteries, long, lasting, price, great]"


In [14]:
id2word = corpora.Dictionary(df['tokens'])

In [19]:
id2word.token2id['bad']

2

In [27]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [38]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 8,
                   passes=10,
                   workers=12
                  )

In [39]:
lda.print_topics()

[(0,
  '0.064*"tablet" + 0.063*"good" + 0.040*"great" + 0.030*"kids" + 0.018*"use" + 0.014*"love" + 0.014*"product" + 0.013*"case" + 0.013*"amazon" + 0.010*"works"'),
 (1,
  '0.041*"easy" + 0.038*"loves" + 0.037*"use" + 0.035*"bought" + 0.031*"old" + 0.024*"year" + 0.022*"tablet" + 0.020*"kindle" + 0.017*"gift" + 0.017*"love"'),
 (2,
  '0.029*"tablet" + 0.023*"kindle" + 0.022*"old" + 0.021*"year" + 0.015*"new" + 0.014*"love" + 0.013*"kids" + 0.013*"friendly" + 0.012*"loves" + 0.011*"son"'),
 (3,
  '0.034*"tablet" + 0.015*"ipad" + 0.014*"great" + 0.013*"good" + 0.013*"kindle" + 0.012*"amazon" + 0.012*"price" + 0.010*"screen" + 0.009*"like" + 0.009*"better"'),
 (4,
  '0.033*"time" + 0.030*"love" + 0.024*"great" + 0.020*"kids" + 0.018*"games" + 0.016*"apps" + 0.015*"like" + 0.015*"free" + 0.012*"parental" + 0.012*"long"'),
 (5,
  '0.028*"great" + 0.027*"tablet" + 0.019*"use" + 0.017*"screen" + 0.016*"kindle" + 0.015*"amazon" + 0.015*"easy" + 0.014*"reading" + 0.013*"books" + 0.012*"read"'

In [41]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[:6]) for t in words]

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
tablet good great kids use love

------ Topic 1 ------
easy loves use bought old year

------ Topic 2 ------
tablet kindle old year new love

------ Topic 3 ------
tablet ipad great good kindle amazon

------ Topic 4 ------
time love great kids games apps

------ Topic 5 ------
great tablet use screen kindle amazon

------ Topic 6 ------
great price good tablet buy bought

------ Topic 7 ------
batteries great price good long work



In [42]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.227383 -0.073505       1        1  24.209560
5      0.022576  0.101159       2        1  16.615265
1      0.169864 -0.027577       3        1  12.319029
3     -0.039803  0.078659       4        1  11.626350
2      0.098749 -0.090933       5        1   9.030695
6     -0.044568  0.040641       6        1   8.983122
0      0.009649  0.064036       7        1   8.898536
4      0.010915 -0.092481       8        1   8.317440, topic_info=           Term         Freq        Total Category  logprob  loglift
19    batteries  6718.000000  6718.000000  Default  30.0000  30.0000
35         good  5090.000000  5090.000000  Default  29.0000  29.0000
18        price  4544.000000  4544.000000  Default  28.0000  28.0000
22        great  9186.000000  9186.000000  Default  27.0000  27.0000
4195     tablet  7491.000000  7491.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
3171      books   142.949692  1583.994995   Topic8  -5.2235   0.0816
108        year   145.707794  1934.654907   Topic8  -5.2044  -0.0993
29       bought   137.567627  3187.614014   Topic8  -5.2619  -0.6561
304        case   121.674583   839.876465   Topic8  -5.3846   0.5549
901        easy   124.101257  3016.468750   Topic8  -5.3649  -0.7039

[672 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
58        1  0.972851    aa
58        4  0.016489    aa
58        5  0.004711    aa
58        7  0.002356    aa
58        8  0.002356    aa
...     ...       ...   ...
2488      5  0.253345    yr
2488      6  0.316028    yr
2488      7  0.023506    yr
2488      8  0.028730    yr
8921      5  0.892333  äôve

[1993 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 6, 2, 4, 3, 7, 1, 5])

In [44]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,8)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

topics_df = pd.DataFrame.from_records(new_distro)
topics_df.columns = topics


In [45]:
topics_df.head()

,tablet good great kids use love,easy loves use bought old year,tablet kindle old year new love,tablet ipad great good kindle amazon,time love great kids games apps,great tablet use screen kindle amazon,great price good tablet buy bought,batteries great price good long work
0,0.010457,0.010444,0.139252,0.010447,0.010446,0.010451,0.304986,0.503517
1,0.020853,0.020843,0.020847,0.020861,0.020863,0.020862,0.020853,0.854018
2,0.031267,0.031298,0.031286,0.031267,0.031264,0.031262,0.295585,0.516771
3,0.020838,0.020838,0.020843,0.020853,0.020847,0.020842,0.020860,0.854079
4,0.020847,0.020838,0.020836,0.020843,0.020847,0.020843,0.020883,0.854062


In [47]:
# most topics are very similar (which is reasonable because most of the products are very similar.
# The topic mostly associated with the reviews is about the price and how long the product lasted.)
topics_df.mean()

tablet good great kids use love          0.106234
easy loves use bought old year           0.142719
tablet kindle old year new love          0.075629
tablet ipad great good kindle amazon     0.085011
time love great kids games apps          0.075018
great tablet use screen kindle amazon    0.116716
great price good tablet buy bought       0.128041
batteries great price good long work     0.259305
dtype: float64

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling